In [1]:
from sys import argv
import gzip
from Bio.SeqIO import parse
import pandas as pd
import re

In [2]:
read1="/public/home/wangxinlong/Project/crispr1/data/20250102ABFC20221434-76_K562/fastq/sgRNA/sg-3_L2_Q0258W0072.R1.fastq.gz"
read2="/public/home/wangxinlong/Project/crispr1/data/20250102ABFC20221434-76_K562/fastq/sgRNA/sg-3_L2_Q0258W0072.R2.fastq.gz"
out="/public/home/wangxinlong/Project/crispr1/result/20250102ABFC20221434-76_K562/sgRNA_identity/"

In [3]:
# read fasta, grna seq:id mapping
grna={}
for rec in parse('/public/home/wangxinlong/Project/crispr1/data/20250102ABFC20221434-76_K562/fasta/sg.fasta','fasta'):
    grna[str(rec.seq)]=str(rec.id)

In [4]:
grna

{'TATTCAGACTCGAAACCGCA': 'GATA1-sg1',
 'GTGTCCTCCACACCAGAATC': 'GATA1-sg2',
 'CCACTCAATGGAGTTACCTG': 'GATA1-sg3',
 'GTGTCCTGCAGCACAAACGG': 'RPS6-sg1',
 'AGTGGTGGGAACGACAAACA': 'RPS6-sg2',
 'TACTTTCTATGAGAAGCGTA': 'RPS6-sg3',
 'TAGGCCTCTGGGCTTTACGG': 'ITGB1-sg1',
 'TGCTGTTCCTTTGCTACGGT': 'ITGB1-sg2',
 'AATGTAACCAACCGTAGCAA': 'ITGB1-sg3',
 'CGAGTGTTATACGCACCGTT': 'NT1',
 'TCGCGCTTGGGTTATACGCT': 'NT2',
 'TGTTAGGCAGATTCCTTATC': 'AAVS'}

In [5]:
n=0 # line number
r=0 # read number

## process read 2, store as read no:grna id
grnas={}
for line in gzip.open(read2,'rt'):
    n+=1
    if n%4==2:
        r+=1
        for s in grna:
            if s in line:
                grnas[r]=[grna[s]]
                break

In [6]:
grnas

{7: ['ITGB1-sg3'],
 8: ['ITGB1-sg1'],
 9: ['ITGB1-sg2'],
 11: ['AAVS'],
 13: ['AAVS'],
 15: ['AAVS'],
 20: ['GATA1-sg2'],
 21: ['ITGB1-sg2'],
 22: ['ITGB1-sg3'],
 23: ['ITGB1-sg3'],
 29: ['ITGB1-sg3'],
 30: ['NT1'],
 33: ['GATA1-sg2'],
 35: ['ITGB1-sg2'],
 36: ['GATA1-sg2'],
 38: ['AAVS'],
 42: ['RPS6-sg1'],
 43: ['ITGB1-sg3'],
 46: ['NT1'],
 47: ['AAVS'],
 48: ['ITGB1-sg2'],
 49: ['AAVS'],
 55: ['GATA1-sg2'],
 56: ['NT1'],
 57: ['NT2'],
 59: ['NT2'],
 60: ['GATA1-sg3'],
 61: ['GATA1-sg3'],
 62: ['ITGB1-sg3'],
 66: ['NT2'],
 67: ['NT2'],
 68: ['NT2'],
 70: ['NT1'],
 71: ['AAVS'],
 72: ['NT2'],
 75: ['ITGB1-sg2'],
 78: ['NT2'],
 80: ['NT2'],
 85: ['NT2'],
 86: ['NT2'],
 89: ['GATA1-sg3'],
 91: ['AAVS'],
 92: ['NT1'],
 93: ['RPS6-sg3'],
 94: ['ITGB1-sg2'],
 96: ['GATA1-sg2'],
 98: ['ITGB1-sg3'],
 99: ['ITGB1-sg3'],
 102: ['NT2'],
 103: ['ITGB1-sg3'],
 104: ['AAVS'],
 105: ['NT2'],
 109: ['ITGB1-sg1'],
 111: ['ITGB1-sg2'],
 112: ['ITGB1-sg1'],
 113: ['ITGB1-sg3'],
 114: ['AAVS'],
 116: ['

In [8]:
## process read1                                
# 3 cell barcodes
cb=pd.read_csv('/public/home/wangxinlong/Project/crispr1/data/20250102ABFC20221434-76_K562/reference/cb.csv')

In [9]:
cb

,b1,b2,b3
0,TGTGTTCGC,TTGTGTTGT,TTGTGGCTG
1,TGTGGCGCC,TTGTGGTAG,TTGTGGAGT
2,TGTCTAGCG,TTGTGCGGA,TTGTGCGAC
3,TGGTTGTCC,TTGTCTGTT,TTGTCTTCA
4,TGGTTCCTC,TTGTCTAAG,TTGTAAGAT
...,...,...,...
379,ACGGATGGT,ACCATCATA,ACACGCATG
380,ACCAGTCTG,ACATATATT,ACACCAGTA
381,ACATTCGGC,ACAGGCACA,ACACCAACT
382,ACATGAGGT,ACACCTGAG,ACACATAGT


In [10]:
# select barcode based on hd
def barcode(bc,b): # string, list of barcodes
    if bc in b:
        return bc
    else:
        for s in b:
            n=0
            for i in range(len(s)): # hamming dist
                if s[i]!=bc[i]:
                    n+=1
            if n==1: # hd=1
                return s
                break

In [11]:
# Filter Read1 sequences using regular expressions
pattern = re.compile(r'(.{9}GTGA.{9}GACA.{9}.{8}(T{0,24}.{1}T{0,24}))')

In [12]:
n = 0  # line number
r = 0  # read number

with gzip.open(read1, 'rt') as f:
    for line in f:
        n += 1
        # Process sequence lines of FASTQ files
        if n % 4 == 2:
            r += 1
            # Check whether the read is in the grnas
            if r in grnas:
                # Match sequences using regular expressions
                match = pattern.search(line)
                if match:
                    matched_seq = match.group(1)

                    # Extract barcodes
                    bc1 = barcode(matched_seq[0:9], cb.b1.tolist()) 
                    bc2 = barcode(matched_seq[13:22], cb.b2.tolist()) 
                    bc3 = barcode(matched_seq[26:35], cb.b3.tolist())

                    # Merge barcodes and process UMI
                    if bc1 is not None and bc2 is not None and bc3 is not None:
                        bc = '_'.join([bc1, bc2, bc3])
                        # Add barcodes and UMIs to the grnas
                        grnas[r] += [bc, matched_seq[35:43]]

In [13]:
grnas

{7: ['ITGB1-sg3', 'GCGTGCATT_TTCGATTGT_TACCGTACT', 'ATCCTAGT'],
 8: ['ITGB1-sg1', 'GATTACGGT_ATGTTAACT_AGGTTGGCT', 'AGGTCGGC'],
 9: ['ITGB1-sg2', 'GTTGGTTCT_CCTACCTTG_TTGGTAACT', 'TGGAAGGA'],
 11: ['AAVS', 'CTTGTCATG_AGTATCATT_TCTTACCAT', 'ATCAATAC'],
 13: ['AAVS', 'GCATCGACG_GAAGTATTG_TTACCGTCA', 'TCGTTAGT'],
 15: ['AAVS', 'GATGTTGGC_CGCGTTCGT_ACTCCTGAC', 'ACCAAATA'],
 20: ['GATA1-sg2', 'ATGCTTACT_TTCTAATCG_ATCCAAGCA', 'GGATAGCA'],
 21: ['ITGB1-sg2', 'CTTGACGGT_GTAGCCTTA_TATGTTCAG', 'GAAGCACA'],
 22: ['ITGB1-sg3', 'GCGTGCATT_TTCGATTGT_TACCGTACT', 'ATCCTAGT'],
 23: ['ITGB1-sg3', 'GGATGAGAG_CTTGATTCT_TACCAGGAC', 'CGATGCTC'],
 29: ['ITGB1-sg3', 'GCACACGAG_TATTATGGT_ACGGTACAG', 'ATAATAAG'],
 30: ['NT1', 'TGTGGCGCC_ATTCTGCAG_TTGGAATCA', 'AACTAATG'],
 33: ['GATA1-sg2', 'GGCGTGCTG_GCTCCGCTG_GCGCGGTGC', 'GCTCGACT'],
 35: ['ITGB1-sg2', 'ATTGCGGAC_TCTGGAATT_ACACCAACT', 'AATGCACG'],
 36: ['GATA1-sg2', 'GTGCAGCCG_CTGTATAAT_ACATTGTGC', 'AGCGGGGT'],
 38: ['AAVS', 'AGAGAATTC_GCCTTGGTA_TGGCTTGTA', 'A

In [14]:
len(grnas)

15763056

In [15]:
## merge reads
a=pd.DataFrame.from_dict(grnas,orient='index', columns=['grna','barcode','umi'])
n1=len(a)

In [16]:
a

,grna,barcode,umi
7,ITGB1-sg3,GCGTGCATT_TTCGATTGT_TACCGTACT,ATCCTAGT
8,ITGB1-sg1,GATTACGGT_ATGTTAACT_AGGTTGGCT,AGGTCGGC
9,ITGB1-sg2,GTTGGTTCT_CCTACCTTG_TTGGTAACT,TGGAAGGA
11,AAVS,CTTGTCATG_AGTATCATT_TCTTACCAT,ATCAATAC
13,AAVS,GCATCGACG_GAAGTATTG_TTACCGTCA,TCGTTAGT
...,...,...,...
32486754,RPS6-sg2,GGCGTGCTG_TTGAGACGT_TATGTTCAG,CCCGTTCC
32486755,GATA1-sg2,None,None
32486756,GATA1-sg2,ATGGTGCAT_ATTGTACAA_TCCATTCAT,GTAACGCT
32486758,NT1,GGAGTCGTG_TTGGCACAG_TATCTGGTC,GGCTGAAG


In [17]:
n1

15763056

In [18]:
# remove umi duplicates: count unique cell barcode, gRNA, umi
# remove none
a=a.value_counts().reset_index()
n2=len(a)

print(read2,'UMI ratio: %.2f'%(n1/n2))

/public/home/wangxinlong/Project/crispr1/data/20250102ABFC20221434-76_K562/fastq/sgRNA/sg-3_L2_Q0258W0072.R2.fastq.gz UMI ratio: 3.89


In [19]:
a

,grna,barcode,umi,count
0,NT2,GTTCCGACG_TATGCTATT_AGCACGGCA,CTGTAATA,371
1,NT1,GTGCGTTCC_ACATATATT_TAGACTGAT,CGCATTAT,367
2,ITGB1-sg3,GTTAGTCCG_TTCTCCGCT_AGTCTACAT,CAACGTCA,360
3,NT1,GTATAGCCT_GCCTGAGTG_TACTTGTCG,GATGGCAA,332
4,NT1,TGTCTAGCG_TGGTTACAA_TCCGGCGTC,GGTGCCAT,331
...,...,...,...,...
4051112,AAVS,GGCGAGATC_GCGTAAGAG_ACCTACTGA,ACTATTGC,1
4051113,ITGB1-sg2,CTTGCTCAC_ACGCCGTTA_AGGCAGGTC,AGGAGGAC,1
4051114,AAVS,GGCGAGATC_GCGCTTACG_TGTACAACG,GAGGTAAC,1
4051115,ITGB1-sg2,CTTGCTCAC_ACGCCGTTA_AGGCAGGTC,AGGTCTGG,1


In [20]:
# sum umi counts
a=a[['barcode','grna']].value_counts().reset_index()

In [21]:
a

,barcode,grna,count
0,ATTGCGGAC_TCTGGAATT_ACACCAACT,ITGB1-sg2,2126
1,TGAAGGACC_AGCTCTAGA_TTGCCTTAC,ITGB1-sg3,884
2,ACTCGCTTG_GCTAACTCT_GCGCCTAGT,NT2,793
3,CCAGGCATT_TTATGCGTT_TATGCAAGT,ITGB1-sg1,779
4,AGGCTTGCG_CTGACCTGT_ACTGAAGCT,NT1,778
...,...,...,...
719662,ACACTAATT_ACTGTATGT_TGGACTTGC,ITGB1-sg2,1
719663,ACACTAATT_ACTGTATGT_TGGACTTGC,NT1,1
719664,ACACTAATT_ACTGTCGAG_ATCTATCAG,AAVS,1
719665,ACACTAATT_ACTGTCGAG_GCTGGTTCA,ITGB1-sg2,1


In [22]:
a.columns=['barcode','sgRNA','counts']

In [23]:
a

,barcode,sgRNA,counts
0,ATTGCGGAC_TCTGGAATT_ACACCAACT,ITGB1-sg2,2126
1,TGAAGGACC_AGCTCTAGA_TTGCCTTAC,ITGB1-sg3,884
2,ACTCGCTTG_GCTAACTCT_GCGCCTAGT,NT2,793
3,CCAGGCATT_TTATGCGTT_TATGCAAGT,ITGB1-sg1,779
4,AGGCTTGCG_CTGACCTGT_ACTGAAGCT,NT1,778
...,...,...,...
719662,ACACTAATT_ACTGTATGT_TGGACTTGC,ITGB1-sg2,1
719663,ACACTAATT_ACTGTATGT_TGGACTTGC,NT1,1
719664,ACACTAATT_ACTGTCGAG_ATCTATCAG,AAVS,1
719665,ACACTAATT_ACTGTCGAG_GCTGGTTCA,ITGB1-sg2,1


In [25]:
# select gRNA [['gRNA','barcode','counts']]
a=a.groupby(['barcode','sgRNA']).sum().sort_values(by='counts',ascending=False)

In [26]:
a

counts
barcode                       sgRNA            
ATTGCGGAC_TCTGGAATT_ACACCAACT ITGB1-sg2    2126
TGAAGGACC_AGCTCTAGA_TTGCCTTAC ITGB1-sg3     884
ACTCGCTTG_GCTAACTCT_GCGCCTAGT NT2           793
CCAGGCATT_TTATGCGTT_TATGCAAGT ITGB1-sg1     779
AGGCTTGCG_CTGACCTGT_ACTGAAGCT NT1           778
...                                         ...
ACACTAATT_ACTGTATGT_TGGACTTGC ITGB1-sg2       1
                              NT1             1
ACACTAATT_ACTGTCGAG_ATCTATCAG AAVS            1
ACACTAATT_ACTGTCGAG_GCTGGTTCA ITGB1-sg2       1
ACACTAATT_ACGTTACTG_AGAGCCGAT RPS6-sg2        1

[719667 rows x 1 columns]

In [27]:
# cell barcodes mapping
m=pd.read_table('/public/home/wangxinlong/Project/crispr1/data/20250102ABFC20221434-76_K562/reference/BD_Enhancedv2_barcode_mapping.tsv',index_col=0)#.astype(int)

In [28]:
m

,id
barcode,
TGTGTTCGC_TTGTGTTGT_TTGTGGCTG,1
TGTGTTCGC_TTGTGTTGT_TTGTGGAGT,2
TGTGTTCGC_TTGTGTTGT_TTGTGCGAC,3
TGTGTTCGC_TTGTGTTGT_TTGTCTTCA,4
TGTGTTCGC_TTGTGTTGT_TTGTAAGAT,5
...,...
ACACTAATT_ACACATTCT_ACACGCATG,56623100
ACACTAATT_ACACATTCT_ACACCAGTA,56623101
ACACTAATT_ACACATTCT_ACACCAACT,56623102


In [29]:
a=a.join(m).dropna()
a['id']=a['id'].astype(int)

In [30]:
a

counts        id
barcode                       sgRNA                      
ATTGCGGAC_TCTGGAATT_ACACCAACT ITGB1-sg2    2126  49431934
TGAAGGACC_AGCTCTAGA_TTGCCTTAC ITGB1-sg3     884   2348176
ACTCGCTTG_GCTAACTCT_GCGCCTAGT NT2           793  55509698
CCAGGCATT_TTATGCGTT_TATGCAAGT ITGB1-sg1     779  48384115
AGGCTTGCG_CTGACCTGT_ACTGAAGCT NT1           778  54068299
...                                         ...       ...
ACACTAATT_ACTGTATGT_TGGACTTGC ITGB1-sg2       1  56616645
                              NT1             1  56616645
ACACTAATT_ACTGTCGAG_ATCTATCAG AAVS            1  56616434
ACACTAATT_ACTGTCGAG_GCTGGTTCA ITGB1-sg2       1  56616375
ACACTAATT_ACGTTACTG_AGAGCCGAT RPS6-sg2        1  56618816

[719667 rows x 2 columns]

In [31]:
a.to_csv('/public/home/wangxinlong/Project/crispr1/result/20250102ABFC20221434-76_K562/sgRNA_identity/sg-3_L2_Q0258W0072_umi_counts.csv')